In [ ]:
import pandas as pd
import os 
import sys
from datetime import datetime, date
from comparison_functions import CompareAccInfo, get_master_info
from extract_functions import acc_info_extract , read_in_email
from progress.bar import IncrementalBar  

arg = sys.argv[1]
print("File Location: "+arg)

#Start Process 
print("Start:{0} {1}".format('Proofing Process',datetime.now().strftime("%H:%M:%S")))
print("Stage 1:{0}".format('Read in Master file'))
#Read in Master File
df = pd.read_csv('C:\\Users\\u336154\\Proofing\\Master_File_PREPAY_RMR.csv',engine='python')
df = df.where(pd.notnull(df), None)

#Get all filenames 
for root, dirs, files in os.walk(arg, topdown=False):
    #for name in files:
        #print(name)
    files = [ fi for fi in files if fi.endswith(".msg") ]

#Generate Hyperlink    
excel_hyperlink = ['=HYPERLINK("'+arg +'/'+ file +'","' + file +'")' for file in files]

print("Stage 2:{0}".format('Compare Emails to Master file'))
#Run proofing process 
output_df = pd.DataFrame(columns = ['Email','Gas_acc_no', 'Elec_acc_no', 'Postcode', 'Tariff', 'Payment_Method','Premise', 'Fuel_Type'])
bar = IncrementalBar("Processing Files:", max=len(files),suffix='%(percent)d%% [elapsed:%(elapsed)ds/ eta:%(eta)ds]')
for counter,file in enumerate(files):
    emails = read_in_email(arg +'/'+file)

    master_info = get_master_info(df,acc_info_extract(emails))
    compare_acc_info = CompareAccInfo(acc_info_extract(emails),master_info)

    output_dict = compare_acc_info.run_all()
    output_list =(list(output_dict.values()))
    output_msg_name = excel_hyperlink[counter]
    output_df.loc[len(output_df)] = [output_msg_name]+output_list

    bar.next()
bar.finish()


summary_flag = output_df.iloc[:,1:].notnull().any(axis=1).tolist()
map_summary_flag = list(map(lambda x: "Fail" if x else "Pass",summary_flag))

output_df['summary']=map_summary_flag
output_df.to_csv("proofing_output_"+date.today().strftime("%d-%m-%Y")+".csv", index=False, encoding='utf-8-sig',mode='w')

print("Stage 3:{0} {1}".format('Output report-',"proofing_output_"+date.today().strftime("%d-%m-%Y")+".csv"))
print("End:{0} {1}".format('Proofing Process',datetime.now().strftime("%H:%M:%S")))
print()
print('Result:' + str(sum(summary_flag)) + '/' + str(len(summary_flag))+ ' failed')